In [ ]:
# %load ../notebooks/init.ipy
%reload_ext autoreload
%autoreload 2

# Builtin packages
from importlib import reload
import logging
import os
from pathlib import Path
import sys
import warnings

# standard secondary packages
import astropy as ap
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.stats
import tqdm.notebook as tqdm

# development packages
import kalepy as kale
import kalepy.utils
import kalepy.plot

# --- Holodeck ----
import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR
import holodeck.gravwaves
import holodeck.evolution
import holodeck.population

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

# Load log and set logging level
log = holo.log
log.setLevel(logging.INFO)

In [ ]:
fname = (
    "/Users/lzkelley/programs/nanograv/holodeck/output/"
    "hard04b_2023-01-23_01_n1000_g100_s40_r50_f40/"
    "sam-lib_hard04b_2023-01-23_01_n1000_g100_s40_r50_f40.hdf5"
)
print(fname)
data = h5py.File(fname, 'r')
print("Keys:", data.keys())
for kk, vv in data.items():
    try:
        print("\t", kk, vv.shape)
    except AttributeError:
        continue
    
print("\nAttributes:")
for kk, vv in data.attrs.items():
    print("\t", kk, vv)
    
print("\nParameters:")
for kk, vv in data['parameters'].items():
    print("\t", kk, vv[:])

In [ ]:
fobs = data['fobs'][:]
xx = fobs * YR
fig, ax = plot.figax()
ff = 0

if False:
    # (nsamples, nfreqs, nreals)
    yy = data['gwb'][:]
    # ==> (nfreqs, nsamples, nreals)
    yy = np.moveaxis(yy, 1, 0)
    yy = np.median(yy, axis=-1)
    yy = yy.reshape(yy.shape[0], -1)
    ax.set(ylim=[1e-17, 1e-13])
    ax.plot(xx, yy, alpha=0.1, color='k', lw=1)

    for amp in [1e-15, 2e-15, 3e-15]:
        plot._draw_plaw(ax, xx, amp, 1.0, color='r')

if True:
    df = np.fabs(xx - 1.0)
    ff = np.argmin(df)
    title = f"Characteristic Strain at ${xx[ff]:.2f} \, \\mathrm{{ yr}}^{{-1}}$"
    ax.set(xscale='linear', yscale='linear', xlim=[-17, -13], title=title)
    yy = data['gwb'][:, ff, :].flatten()
    yy = np.log10(yy[yy > 0.0])
    kale.dist1d(yy, ax=ax, density=True, carpet=True)

plt.show()

In [ ]:
num_label = 5
xx = data['fobs'][:] * YR
gwb = data['gwb'][:]
spars = data['sample_params'][:]
print(gwb.shape, spars.shape)
# print(f"{len(data.attrs['param_names'])=}")

spars = spars.T
for ii, pname in enumerate(data.attrs['param_names']):
    fig, ax = plot.figax(xlabel="Frequency [1/yr]", ylabel="Characteristic Strain")
    plot._twin_hz(ax)

    pvals = data['parameters'][pname][:]
    print(ii, pname, pvals[:3], "...", pvals[-3:])
    sv = spars[ii].copy()
    sort_idx = np.argsort(sv)
    sv = sv[sort_idx]
    lo = 0

    num_pvals = len(pvals)
    colors = mpl.cm.get_cmap('Spectral')(np.linspace(0.1, 0.9, num_pvals))

    for jj, pv in enumerate(pvals):
        # Make sure there are only `num_label` labeled lines on the plot
        label = (jj % (num_pvals//num_label) == 0) or (num_pvals <= num_label)

        # This boolean array is False until the first value ABOVE this bin
        hi = (sv > pv)
        # Find the index of the first value ABOVE of this bin
        hi = np.argmax(hi)
        # Make sure ANY of the values are above this bin
        hi = hi if (sv[hi] > pv) else None
        # Select the values since the last bin, up to the next bin
        cut = slice(lo, hi)
        # Select the GWB spectra in this bin, shape (samples, freqs, reals)
        yy = gwb[sort_idx][cut]
        # convert to shape (freqs, samples, reals)
        yy = np.moveaxis(yy, 1, 0)
        # convert to shape (freqs, samples*reals)
        yy = yy.reshape(yy.shape[0], -1)

        med = np.median(yy, axis=-1)
        conf = np.percentile(yy, [25, 75], axis=-1)
        label = f"{pv:.1e}" if label else ""
        ax.plot(xx, med, color=colors[jj], label=label)
        ax.fill_between(xx, *conf, color=colors[jj], alpha=0.2)
        
        # set the lo-end of the next bin, to be the hi-end of this bin
        lo = hi

    ax.legend(title=pname)    
    

In [ ]:
# kale.corner(data['params'][:].T)